In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [14]:
acs_data = pd.read_csv('data/ACS Data.csv')
acs_data.head()

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,Summary Level,Geographic Component,File identification,Logical Record Number,State (FIPS Code),...,Occupied Housing Units: 3 Rooms,Occupied Housing Units: 4 Rooms,Occupied Housing Units: 5 Rooms,Occupied Housing Units: 6 Rooms,Occupied Housing Units: 7 Rooms,Occupied Housing Units: 8 Rooms,Occupied Housing Units: 9 or More Rooms,Median Year Structure Built,Median Gross Rent,Owner-Occupied Housing Units
0,1.700000e+11,15000US170310101001,Block Group 1,"Block Group 1, Census Tract 101, Cook County, ...",il,150,0,ACSSF,9336,17,...,20,35,50,15,35,0,47,0.0,873.0,117
1,1.700000e+11,15000US170310101002,Block Group 2,"Block Group 2, Census Tract 101, Cook County, ...",il,150,0,ACSSF,9337,17,...,238,310,227,77,23,0,57,0.0,799.0,53
2,1.700000e+11,15000US170310101003,Block Group 3,"Block Group 3, Census Tract 101, Cook County, ...",il,150,0,ACSSF,9338,17,...,216,338,161,97,7,14,41,0.0,1071.0,327
3,1.700000e+11,15000US170310102011,Block Group 1,"Block Group 1, Census Tract 102.01, Cook Count...",il,150,0,ACSSF,9339,17,...,89,206,189,90,0,10,33,1962.0,1097.0,224
4,1.700000e+11,15000US170310102012,Block Group 2,"Block Group 2, Census Tract 102.01, Cook Count...",il,150,0,ACSSF,9340,17,...,221,412,442,125,119,0,19,1945.0,1152.0,353


In [15]:
acs_features = acs_data.iloc[:, [0, 1, 11, 12, 13, 31, 32, 38, 39, 40, 50, 51, 52]].copy()
acs_features.head()

,FIPS,Geographic Identifier,Census Tract,Block Group,Total Population,Total Population: White Alone,Total Population: Black or African American Alone,Average Household Size,Median Household Income (In 2019 Inflation Adjusted Dollars),Occupied Housing Units,Median Year Structure Built,Median Gross Rent,Owner-Occupied Housing Units
0,1.700000e+11,15000US170310101001,10100,1,461,265,108,1.95,NaN,236,0.0,873.0,117
1,1.700000e+11,15000US170310101002,10100,2,2424,746,1543,2.26,21827.0,1054,0.0,799.0,53
2,1.700000e+11,15000US170310101003,10100,3,1714,1137,427,1.50,54297.0,1073,0.0,1071.0,327
3,1.700000e+11,15000US170310102011,10201,1,1706,479,745,2.30,42778.0,712,1962.0,1097.0,224
4,1.700000e+11,15000US170310102012,10201,2,3925,2131,1180,2.69,39535.0,1424,1945.0,1152.0,353


In [16]:
zipfile = 'data/tl_2020_17_bg.zip'
il_census_bgs = gpd.read_file(zipfile)
il_census_bgs.to_crs(epsg=4326, inplace=True)

chicago_boundaries = gpd.read_file('data/Boundaries - City.geojson')

chicago_census_bgs = gpd.sjoin(il_census_bgs, chicago_boundaries, op='within')
chicago_census_bgs.loc[:, ['GEOID', 'geometry']].head()

,GEOID,geometry
113,170310208022,"POLYGON ((-87.69952 41.99398, -87.69927 41.993..."
114,170310627002,"POLYGON ((-87.66846 41.93423, -87.66798 41.934..."
123,170317104004,"POLYGON ((-87.67091 41.75743, -87.67083 41.757..."
124,170315611005,"POLYGON ((-87.77815 41.79242, -87.77791 41.792..."
138,170318315001,"POLYGON ((-87.79080 41.93824, -87.79077 41.938..."


In [17]:
acs_features["GEOID"] = acs_features["Geographic Identifier"].map(lambda x: str(x)[-12:])
geo_census_features = (gpd.GeoDataFrame(acs_features.merge(chicago_census_bgs.loc[:, ['GEOID', 'geometry']],
                                                           on="GEOID", how="inner"))) 
geo_census_features.head()

,FIPS,Geographic Identifier,Census Tract,Block Group,Total Population,Total Population: White Alone,Total Population: Black or African American Alone,Average Household Size,Median Household Income (In 2019 Inflation Adjusted Dollars),Occupied Housing Units,Median Year Structure Built,Median Gross Rent,Owner-Occupied Housing Units,GEOID,geometry
0,1.700000e+11,15000US170310102011,10201,1,1706,479,745,2.30,42778.0,712,1962.0,1097.0,224,170310102011,"POLYGON ((-87.68408 42.01515, -87.68387 42.015..."
1,1.700000e+11,15000US170310102012,10201,2,3925,2131,1180,2.69,39535.0,1424,1945.0,1152.0,353,170310102012,"POLYGON ((-87.68039 42.01360, -87.68028 42.013..."
2,1.700000e+11,15000US170310102022,10202,2,1591,659,598,2.47,47024.0,620,1946.0,1032.0,117,170310102022,"POLYGON ((-87.67599 42.01569, -87.67585 42.015..."
3,1.700000e+11,15000US170310103004,10300,4,1619,1177,224,2.04,39004.0,783,0.0,1072.0,291,170310103004,"POLYGON ((-87.67033 42.01683, -87.67033 42.016..."
4,1.700000e+11,15000US170310105011,10501,1,935,469,195,1.81,43516.0,509,0.0,973.0,91,170310105011,"POLYGON ((-87.66582 42.01035, -87.66515 42.010..."


In [18]:
test = geo_census_features.copy()
print('Gross Rent NA %: {:.1f}'.format(len(test.loc[test['Median Gross Rent'].isna()]) / len(test) * 100))
print('Median Home Age NA: % {:.1f}'.format(len(test.loc[test['Median Year Structure Built'].eq(0)]) / len(test) * 100))
print('Median NA %: {:.1f}'.format(len(test.loc[test['Median Household Income (In 2019 Inflation Adjusted Dollars)'].isna()]) / len(test) * 100))
print('Total Pop = 0 %: {:.1f}'.format(len(test.loc[test['Total Population'].eq(0)]) / len(test) * 100))
print('Occupied Units = 0 %: {:.1f}'.format(len(test.loc[test['Occupied Housing Units'].eq(0)]) / len(test) * 100))
print('Owner Occupied Units = 0 %: {:.1f}'.format(len(test.loc[test['Owner-Occupied Housing Units'].eq(0)]) / len(test) * 100))
print('White Alone = 0 %: {:.1f}'.format(len(test.loc[test['Total Population: White Alone'].eq(0)]) / len(test) * 100))
print('Black Alone = 0 %: {:.1f}'.format(len(test.loc[test['Total Population: Black or African American Alone'].eq(0)]) / len(test) * 100))


Gross Rent NA %: 9.0
Median Home Age NA: % 48.3
Median NA %: 5.8
Total Pop = 0 %: 0.1
Occupied Units = 0 %: 0.2
Owner Occupied Units = 0 %: 0.9
White Alone = 0 %: 11.4
Black Alone = 0 %: 15.0


In [19]:
geo_census_features['Percent White'] = np.divide(geo_census_features['Total Population: White Alone'],
                                          geo_census_features['Total Population'])
geo_census_features['Percent Non-White'] = np.subtract(1, geo_census_features['Percent White'])
geo_census_features['Percent African American'] = np.divide(geo_census_features['Total Population: Black or African American Alone'],
                                                     geo_census_features['Total Population'])
geo_census_features['Median Year Structure Built'] = np.where((geo_census_features['Median Year Structure Built'].eq(0)), 
                                                       np.nan, geo_census_features['Median Year Structure Built'])
geo_census_features['Percent Owner Occupied'] = np.divide(geo_census_features['Owner-Occupied Housing Units'],
                                                   geo_census_features['Occupied Housing Units'])
geo_census_features['Median Year Structure Built'].median()

1955.0

In [20]:
geo_census_features.rename(mapper={'Total Population': 'tot_pop', 
                                   'Total Population: White Alone': 'tot_white', 
                                   'Total Population: Black or African American Alone': 'tot_black', 
                                   'Average Household Size': 'hh_size',
                                   'Median Household Income (In 2019 Inflation Adjusted Dollars)': 'med_income', 
                                   'Occupied Housing Units': 'occ_units',
                                   'Median Year Structure Built': 'med_built_yr',
                                   'Median Gross Rent': 'med_rent',
                                   'Owner-Occupied Housing Units': 'oo_hsng_units',
                                   'Percent White': 'perc_white', 
                                   'Percent Non-White': 'perc_non_white',
                                   'Percent African American': 'perc_black',
                                   'Percent Owner Occupied': 'perc_owner_occ'}, 
                  axis='columns', inplace=True)

In [21]:
geo_census_features.to_file("data/geo_census.shp")

<ipython-input-21-28f77e3a32f2>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_census_features.to_file("data/geo_census.shp")
